In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import binance_framework as bf
from collections import deque
import random

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

In [3]:
CURRENCIES = ['ETHUSDT']
SEQUENCE_LEN = 128
FUTURE_PERIOD_PREDICT = 5
RATIO_TO_PREDICT = 'ETHUSDT'
EPOCHS = 20
BATCH_SIZE = 128
NAME = 'SEQ-'+ str(SEQUENCE_LEN)+'-FUTPRED-'+str(FUTURE_PERIOD_PREDICT)+'-PRED-'+str(time.time())

In [7]:
coins = bf.get_data_by_intervals(300,CURRENCIES,bf.INTERVAL_1MINUTE)

In [8]:
coins.to_csv('eth_1min_300.csv',index=False)

In [10]:
coins  = pd.read_csv('eth_1min_300.csv')

In [11]:
coins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149701 entries, 0 to 149700
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   ETHUSDT Open Time                     149701 non-null  int64  
 1   ETHUSDT Open                          149701 non-null  float64
 2   ETHUSDT High                          149701 non-null  float64
 3   ETHUSDT Low                           149701 non-null  float64
 4   ETHUSDT Close                         149701 non-null  float64
 5   ETHUSDT Volume                        149701 non-null  float64
 6   ETHUSDT Close Time                    149701 non-null  int64  
 7   ETHUSDT Quote asset volume            149701 non-null  float64
 8   ETHUSDT n_trades                      149701 non-null  int64  
 9   ETHUSDT Taker buy base asset volume   149701 non-null  float64
 10  ETHUSDT Taker buy quote asset volume  149701 non-null  float64
 11  

In [12]:
coins.head()

,ETHUSDT Open Time,ETHUSDT Open,ETHUSDT High,ETHUSDT Low,ETHUSDT Close,ETHUSDT Volume,ETHUSDT Close Time,ETHUSDT Quote asset volume,ETHUSDT n_trades,ETHUSDT Taker buy base asset volume,ETHUSDT Taker buy quote asset volume,ETHUSDT Ignore
0,1574349360000,160.69,160.85,160.22,160.42,1810.54483,1574349419999,290646.952740,438,1297.35970,208236.038870,0
1,1574349420000,160.38,160.78,160.25,160.75,1720.26172,1574349479999,276106.345627,350,1372.16630,220235.288980,0
2,1574349480000,160.70,160.78,160.34,160.37,1395.59348,1574349539999,224053.323758,307,422.60231,67836.519003,0
3,1574349540000,160.37,160.52,160.30,160.36,1437.55787,1574349599999,230559.363933,311,1194.79611,191624.745779,0
4,1574349600000,160.36,160.93,160.21,160.91,1226.09076,1574349659999,196834.782570,416,957.50325,153716.993099,0


In [13]:
bf.show_time_skips(coins, CURRENCIES)

Nanosec default difference:  60000
ETHUSDT  Time difference:  7260000
between indexes:  4963 , 4964
gap starts at:  1574647140000
gap ends at:    1574654400000


ETHUSDT  Time difference:  3660000
between indexes:  114283 , 114284
gap starts at:  1581213540000
gap ends at:    1581217200000


ETHUSDT  Time difference:  21300000
between indexes:  129199 , 129200
gap starts at:  1582112100000
gap ends at:    1582133400000


ETHUSDT  Time difference:  7740000
between indexes:  148871 , 148872
gap starts at:  1583313660000
gap ends at:    1583321400000




In [14]:
coins = coins[4964:114283]

In [15]:
bf.show_time_skips(coins, CURRENCIES)

Nanosec default difference:  60000


In [ ]:
coins.set_index(CURRENCIES[0]+' Close Time', inplace=True)

In [ ]:
coins.drop([(CURRENCIES[0]+' Open Time'), (CURRENCIES[0]+' Ignore')], axis=1, inplace=True)

In [ ]:
coins.head()

In [ ]:
coins.info()

In [ ]:
coins.index[0]

In [ ]:
print(datetime.fromtimestamp(coins.index[0]/1000))
print(datetime.fromtimestamp(coins.index[-1]/1000))

In [16]:
def classify(current_price, future_price):
    if future_price > current_price:
        return 1
    return 0

In [17]:
coins['Future Value'] = coins[RATIO_TO_PREDICT+' Close'].shift(-FUTURE_PERIOD_PREDICT)

In [18]:
coins['Target'] = list(map(classify, coins[CURRENCIES[0]+' Close'], coins['Future Value']))

In [19]:
coins.dropna(inplace=True)
coins.reset_index(inplace=True)

In [20]:
#gotta split in train and test data
val_len = int(len(coins)*0.2)
val_len

21862

In [21]:
validation_data = coins[(len(coins)-val_len):]
validation_data.reset_index(inplace=True)

In [22]:
coins = coins[:(len(coins)-val_len)]
coins.reset_index(inplace=True)

In [23]:
bf.show_time_skips(coins, CURRENCIES)

Nanosec default difference:  60000


In [24]:
X_train = coins.drop(['Future Value', 'Target'], axis=1)
y_train = coins['Target']

In [25]:
X_test = validation_data.drop(['Future Value', 'Target'], axis=1)
y_test = validation_data['Target']

In [26]:
from sklearn.preprocessing import MinMaxScaler

In [27]:
scaler = MinMaxScaler()

In [28]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)